# Import neccessary modules

In [7]:
import json
import os
import gradio as gr
from dotenv import load_dotenv
# from swarmauri.community.tools.concrete.ZapierHookTool import ZapierHookTool as Tool
from swarmauri.standard.llms.concrete.GroqToolModel import GroqToolModel
from swarmauri.standard.conversations.concrete.Conversation import Conversation
from swarmauri.standard.toolkits.concrete.Toolkit import Toolkit
from swarmauri.standard.agents.concrete.ToolAgent import ToolAgent
import json
import requests
from typing import Dict, List, Literal
from swarmauri.standard.tools.base.ToolBase import ToolBase
from swarmauri.standard.tools.concrete.Parameter import Parameter
from pydantic import Field


In [8]:
"""
- Zapier's webhooks do not require authentication for basic usage,
    as they are designed to be secure via their unique URLs.
- The webhook URL (https://hooks.zapier.com/hooks/catch/{zap_id}/) includes
    a randomly generated identifier that is private to you,
    ensuring that only those with the specific URL can trigger the Zap.

Reference
---------------
- https://www.switchlabs.dev/resources/locating-your-webhook-url-in-zapier
- https://zapier.com/apps/webhook/integrations
- https://zapier.com/engineering/webhook-design/

"""


class ZapierHookTool(ToolBase):
    version: str = "1.0.0"
    parameters: List[Parameter] = Field(
        default_factory=lambda: [
            Parameter(
                name="payload",
                type="string",
                description="A Payload to send when triggering the Zapier webhook",
                required=True,
            )
        ]
    )

    name: str = "ZapierTool"
    description: str = "Tool to authenticate with Zapier and execute zaps."
    type: Literal["ZapierHookTool"] = "ZapierHookTool"

    zap_url: str

    def execute_zap(self, payload: str) -> str:
        """Execute a zap with given payload.

        Args:
            payload (str): The data payload to send to the Zap.

        Returns:
            str: The response from Zapier API.
        """

        response = requests.post(
            self.zap_url,
            json={"data": payload},
        )
        # Checking the HTTP response status for success or failure
        if response.status_code == 200:
            return json.dumps(response.json())
        else:
            response.raise_for_status()  # This will raise an error for non-200 responses

    def __call__(self, payload: str) -> Dict[str, str]:
        """
        Enable the tool to be called directly with the given payload and return the zap response.

        Parameters:
        payload (str): The input string to be processed by the tool.

        Returns:
        Dict[str, str]: A dictionary with a single key "zap_response", containing the result of executing the zap.

        Example:
        >>> tool = YourToolClass()
        >>> result = tool("some payload")
        >>> print(result)
        {'zap_response': 'processed result'}
        """
        return {"zap_response": self.execute_zap(payload)}


# Load and instatiate needed variables

In [9]:
load_dotenv()
zap_url = os.getenv('ZAP_URL')

# Load tool and payload

In [11]:
tool = ZapierHookTool(zap_url=zap_url)

# Instatiate Tool Model LLM

In [12]:
# Fetch the API key from environment variables or define it directly (Not recommended for production)
API_KEY = os.getenv('GROQ_API_KEY')

# Initialize the GroqToolModel
tool_model = GroqToolModel(api_key=API_KEY)

# Create a ToolAgent with the GroqToolModel, Conversation, and Toolkit
toolkit = Toolkit(tools={"ZapierTool": tool})
tool_agent = ToolAgent(
    llm=tool_model,
    conversation=Conversation(),
    toolkit=toolkit
)

# Instatiate the Interface

In [13]:
# Define the function to be executed for the gradio interface
def tool_agent_interface(input_text, history):
    result = tool_agent.exec(input_text)
    return result

demo = gr.ChatInterface(tool_agent_interface)

if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://1254fbedea40ec2425.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
